
---

## 실습: 웹 스크래핑과 API를 사용한 데이터 추출  
**예상 소요 시간: 30분**  

### 소개  
웹 스크래핑은 웹 페이지에서 관련 데이터를 추출하는 데 사용됩니다. 공개된 웹 페이지에서 특정 데이터를 가져와야 할 때, 웹 스크래핑은 데이터를 쉽게 추출할 수 있도록 해 줍니다. 그러나 웹 스크래핑을 사용하려면 HTML 페이지 구조에 대한 기본 지식이 필요합니다.  

이 실습에서는 웹 페이지의 HTML 코드를 분석하고 Python을 사용하여 필요한 정보를 추출하는 방법을 배우게 됩니다.  

---

### **목표**  
이 실습을 완료하면 다음을 수행할 수 있습니다:  

- `requests` 및 `BeautifulSoup` 라이브러리를 사용하여 웹 페이지의 내용을 추출하기  
- 웹 페이지의 HTML 코드를 분석하여 관련 정보를 찾기  
- 필요한 정보를 추출하고 원하는 형식으로 저장하기  

---

### **시나리오**  
당신은 멀티플렉스 영화관 관리 회사에 고용되어, 아래 공유된 웹 링크에서 **평균 평점이 가장 높은 50편의 영화** 정보를 추출해야 합니다.  

🔗 링크:  
[https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films](https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films)  

---

### **요구사항**  
추출해야 할 정보는 다음과 같습니다:  

- **평균 순위 (Average Rank)**  
- **영화 제목 (Film)**  
- **제작 연도 (Year)**  

---

### **작업 지시**  
- `webscraping_movies.py`라는 **Python 스크립트**를 작성하여 위 링크에서 정보를 추출하고, 결과를 `top_50_films.csv`라는 CSV 파일로 저장하세요.  
- 같은 정보를 **Movies.db**라는 데이터베이스에 저장하고, 테이블 이름은 **Top_50**으로 설정하세요.  

---

이제 Python과 BeautifulSoup을 사용한 **웹 스크래핑 프로젝트**를 시작해 보세요! 🚀


---

## **초기 단계**  
이 실습을 진행하려면 다음 라이브러리가 필요합니다:  

1️⃣ **pandas** 라이브러리: 데이터 저장 및 조작에 사용  
2️⃣ **BeautifulSoup** 라이브러리: HTML 문서를 해석하는 데 사용  
3️⃣ **requests** 라이브러리: 웹 페이지와 통신하는 데 사용  
4️⃣ **sqlite3** 라이브러리: 데이터베이스 인스턴스를 생성하는 데 사용  

---

### **라이브러리 설치 안내**  
- `requests`와 `sqlite3`는 **Python3**에 기본 포함되어 있습니다.  
- 그러나 **pandas**와 **BeautifulSoup** 라이브러리는 직접 설치해야 합니다.  

터미널 창에서 다음 명령어를 실행하여 필요한 라이브러리를 설치하세요:  

```bash
python3.11 -m pip install pandas  
python3.11 -m pip install bs4  
```

✅ **명령어 실행 완료!**  

---

### **Python 파일 생성**  
이제 `/home/project/` 경로에 **webscraping_movies.py**라는 새 파일을 만드세요.  
해당 파일에 모든 코드를 작성하면 됩니다.  

💻 **코딩을 시작할 준비가 되셨나요?** 😃  
이제 Python을 사용해 웹 스크래핑을 시작하세요! 🚀

**{영한} 번역: Code setup**  

---

## **코드 설정**  

### **1️⃣ 라이브러리 가져오기**  
Python 스크립트를 생성할 때, 먼저 필요한 라이브러리를 가져와야 합니다.  

💻 **webscraping_movies.py** 파일에 아래 코드를 추가하여 네 가지 라이브러리를 가져오세요:  

```python
import requests  
import sqlite3  
import pandas as pd  
from bs4 import BeautifulSoup  
```

---

### **2️⃣ 주요 변수 초기화**  
코드의 시작 부분에서 몇 가지 필수 변수를 선언해야 합니다.  

필요한 정보는 다음과 같습니다:  

- **URL**: 데이터를 가져올 웹 페이지의 링크  
- **데이터베이스 이름**: 정보를 저장할 DB 이름  
- **CSV 경로**: 추출한 데이터를 저장할 CSV 파일 경로  
- **테이블 이름**: 데이터베이스 내 테이블 이름  
- **데이터프레임**: `pandas`를 사용해 데이터를 저장할 표 형식  
- **카운터**: 상위 50개의 결과를 추출하기 위한 반복문 카운터  

아래 코드를 **webscraping_movies.py**에 추가하세요:  

```python
url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'  
db_name = 'Movies.db'  
table_name = 'Top_50'  
csv_path = '/home/project/top_50_films.csv'  
df = pd.DataFrame(columns=["Average Rank", "Film", "Year"])  
count = 0  
```

---

### **3️⃣ 웹 페이지 로드 및 파싱**  
이제 웹 페이지에서 필요한 정보를 가져오기 위해 전체 HTML 문서를 Python에서 불러와야 합니다.  
먼저 **requests.get().text**를 사용하여 웹 페이지의 HTML을 로드한 후, **BeautifulSoup**을 사용하여 HTML을 파싱합니다.  

아래 코드를 **webscraping_movies.py**에 추가하세요:  

```python
html_page = requests.get(url).text  
data = BeautifulSoup(html_page, 'html.parser')  
```

---

### **4️⃣ 파일 저장**  
코드 작업이 끝나면 **Ctrl+S**를 눌러 **webscraping_movies.py** 파일을 저장하세요.  

이제 웹 스크래핑 작업을 시작할 준비가 완료되었습니다! 🎉  
**다음 단계로 진행해 봅시다! 🚀**


---

## **HTML 코드에서 관련 정보 분석하기**  

### **1️⃣ 웹 페이지 열기**  
브라우저에서 주어진 웹 페이지를 열고, 필요한 **테이블(Table)**을 찾을 때까지 아래로 스크롤합니다.  

🔗 링크:  
[https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films](https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films)

---

### **2️⃣ 테이블 요소 검사**  
1. 테이블을 찾은 후, 해당 테이블에서 **마우스 오른쪽 버튼**을 클릭합니다.  
2. 나타나는 메뉴에서 **Inspect(검사)**를 클릭합니다.  

이렇게 하면 개발자 도구가 열리며, HTML 코드 내에서 테이블 요소를 확인할 수 있습니다.  
이 단계는 테이블의 **클래스 이름**이나 **태그 구조**를 파악하여 웹 스크래핑을 할 때 정확히 어떤 정보를 추출할지를 결정하는 데 중요합니다.  
<img src='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/images/inspect.png' />
✅ **HTML 구조를 분석한 후, 필요한 정보를 추출하기 위한 코드 작성으로 진행하세요!**  

**다음 단계에서 테이블의 구조를 확인하고 코드를 작성해 보겠습니다! 🚀**


---

## **HTML 코드에서 테이블 시작 부분 확인하기**  

### **1️⃣ 테이블 정의 위치로 이동**  
브라우저의 **검사(Inspect)** 도구를 사용하면 웹 페이지의 **HTML 코드**가 열리며, 바로 **테이블 정의(tbody 태그)** 시작 부분으로 이동하게 됩니다.  

---

### **2️⃣ 테이블 강조 표시 확인하기**  
1. HTML 코드에서 **`<tbody>` 태그**를 찾으세요.  
2. 마우스 포인터를 **`<tbody>` 태그** 위에 가져가면, 페이지의 테이블 섹션이 **하이라이트(강조 표시)** 됩니다.  
   - 이 단계는 테이블이 페이지에서 정확히 어디에 위치하는지 확인하고, 올바른 데이터를 추출하는 데 도움이 됩니다.

---
<img src='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/images/html_tbody.png' />

### **🔍 팁**  
- **`<tbody>` 태그**는 테이블 본문을 정의하는 태그로, 웹 스크래핑 시 **데이터가 포함된 행(tr)과 열(td)**을 탐색할 때 중요합니다.  
- 이제 테이블 구조를 더 자세히 분석하여 원하는 정보를 추출할 준비를 마쳤습니다! 🎉

**다음 단계로 넘어가서 데이터를 추출하는 코드를 작성해 봅시다! 🚀**


---

## **테이블의 HTML 구조 이해하기**  

### **1️⃣ 테이블의 행과 열 구조**  
테이블에서 모든 **행(row)**은 **`<tr>` 객체**로 정의되어 있습니다.  
- **`<tr>`** 태그를 클릭하면, 해당 행에 포함된 데이터가 **`<td>` 객체(열 데이터)**로 저장되어 있는 것을 확인할 수 있습니다.  

예를 들어, 각 **`<tr>`** 안에 다음과 같은 데이터가 포함됩니다:  

```html
<tr>
    <td>1</td>
    <td>The Godfather</td>
    <td>1972</td>
</tr>
```

---

### **2️⃣ 필요한 데이터 확인**  
당신은 **첫 번째 테이블**의 **상위 3개의 헤더**에 해당하는 정보를 추출해야 합니다:  
1. **Average Rank**  
2. **Film**  
3. **Year**  

---

### **3️⃣ 테이블 식별**  
중요한 점은, 이 테이블이 **페이지에서 첫 번째 테이블**이라는 것입니다.  
따라서 정보를 추출할 때 반드시 **페이지의 첫 번째 테이블**을 선택해야 합니다.  

---

### **🔍 팁**  
Python의 **BeautifulSoup** 라이브러리를 사용할 때, 첫 번째 테이블을 다음과 같이 선택할 수 있습니다:

```python
table = data.find('table')  # 첫 번째 테이블 선택
rows = table.find_all('tr')  # 모든 행(tr) 가져오기
```

이제 데이터를 분석하고, 상위 50개의 영화 정보를 추출할 준비가 되었습니다! 🎬  
다음 단계에서는 데이터를 효율적으로 저장하는 코드를 작성해 보겠습니다! 🚀



---

## **필요한 정보 스크래핑하기**  

### **1️⃣ 테이블의 행 데이터 가져오기**  
웹 페이지에서 **필요한 정보를 추출하기 위한 반복문**을 작성해야 합니다.  
먼저, **BeautifulSoup** 객체를 사용하여 **`find_all()`** 함수로 모든 테이블의 본문을 가져오고, 첫 번째 테이블의 모든 **행(tr)**을 가져옵니다.

💻 **다음 코드를 추가하세요:**  
```python
tables = data.find_all('tbody')  
rows = tables[0].find_all('tr')  
```

- **`tables`** 변수: 페이지에 있는 모든 테이블의 본문을 가져옴  
- **`rows`** 변수: 첫 번째 테이블의 모든 행을 가져옴  

---

### **2️⃣ 반복문으로 데이터 추출하기**  
이제 **반복문(for loop)**을 사용하여 각 행에서 데이터를 추출하고, 상위 50개의 결과를 **데이터프레임(df)**에 추가합니다.

💻 **다음 코드를 작성하세요:**  
```python
for row in rows:  
    if count < 50:  
        col = row.find_all('td')  
        if len(col) != 0:  
            data_dict = {"Average Rank": col[0].contents[0],  
                         "Film": col[1].contents[0],  
                         "Year": col[2].contents[0]}  
            df1 = pd.DataFrame(data_dict, index=[0])  
            df = pd.concat([df, df1], ignore_index=True)  
            count += 1  
    else:  
        break  
```

---

### **3️⃣ 코드 작동 방식 설명**  

1️⃣ **`for row in rows:`**  
- 행 데이터를 반복적으로 가져옵니다.

2️⃣ **`if count < 50:`**  
- 루프 카운터를 확인하여 **상위 50개의 데이터**만 추출합니다.

3️⃣ **`col = row.find_all('td')`**  
- 각 행에서 **모든 열(td)** 데이터를 추출합니다.

4️⃣ **`if len(col) != 0:`**  
- 열 데이터가 없는 경우를 제외합니다. (병합된 행처럼 보이지 않는 데이터가 있을 수 있기 때문입니다.)

5️⃣ **`data_dict = {...}`**  
- 데이터 사전을 생성하여, 각 열에서 추출한 데이터를 **"Average Rank"**, **"Film"**, **"Year"** 키에 저장합니다.

6️⃣ **`pd.DataFrame()`**  
- 사전을 **데이터프레임**으로 변환하고 기존 데이터프레임과 **병합(pd.concat)**합니다.

7️⃣ **`count += 1`**  
- 반복문이 실행될 때마다 카운터를 증가시킵니다.

8️⃣ **`break`**  
- 카운터가 50에 도달하면 루프를 종료합니다.

---

### **4️⃣ 데이터 출력**  
데이터프레임의 내용을 확인하려면 다음 코드를 추가하세요:  

```python
print(df)  
```

---

### **5️⃣ 파일 저장**  
작업이 끝나면 **Ctrl+S**를 눌러 **webscraping_movies.py** 파일을 저장하세요.

✅ 이제 모든 데이터가 **데이터프레임**에 수집되었으며, 다음 단계로 데이터 저장 및 활용을 진행할 준비가 되었습니다! 🚀  
다음 단계로 이동해 보세요! 🎉



---

## **데이터 저장하기**  

### **1️⃣ 데이터프레임을 CSV 파일로 저장**  
데이터프레임을 **CSV 파일**로 저장하려면 다음 명령어를 사용하세요:

💻 **코드:**  
```python
df.to_csv(csv_path)
```

- **`csv_path`**는 이전에 선언한 **CSV 파일 경로** 변수입니다.  
- 이 명령어는 데이터를 **`/home/project/top_50_films.csv`** 파일로 저장합니다.

---

### **2️⃣ 데이터베이스에 데이터 저장하기**  
데이터를 **데이터베이스(DB)**에 저장하려면 먼저 데이터베이스에 연결하고, 데이터를 테이블로 저장한 후 연결을 닫아야 합니다.  

💻 **다음 코드를 작성하세요:**  
```python
conn = sqlite3.connect(db_name)  
df.to_sql(table_name, conn, if_exists='replace', index=False)  
conn.close()
```

- **`conn = sqlite3.connect(db_name)`**: 데이터베이스에 연결합니다.  
- **`df.to_sql()`**: 데이터프레임을 데이터베이스의 **테이블(Top_50)**로 저장합니다.  
  - **`if_exists='replace'`**: 테이블이 이미 존재하면 덮어씁니다.  
  - **`index=False`**: 데이터프레임의 인덱스는 저장하지 않습니다.  
- **`conn.close()`**: 데이터베이스 연결을 닫습니다.

---

### **3️⃣ 파일 저장**  
작업이 끝나면 **Ctrl+S**를 눌러 **webscraping_movies.py** 파일을 저장하세요.

---

### **4️⃣ 완료!** ✅  
이제 데이터가 **CSV 파일**과 **데이터베이스(Top_50 테이블)**에 저장되었습니다.

---

### **💡 추가 정보:**  
이제 SQL 쿼리를 사용하여 **Movies.db**에서 필요한 정보를 검색할 수 있습니다.  
데이터베이스 활용 방법은 코스의 **후속 강의**에서 다룰 예정입니다.  

🎉 **축하합니다!** 모든 데이터 스크래핑 및 저장 작업을 성공적으로 완료했습니다! 🚀

---

## **코드 실행하기**  

### **1️⃣ 코드 실행 명령어**  
터미널 창에서 아래 명령어를 사용하여 코드를 실행하세요.  
현재 디렉터리가 **`/home/project/`**인지 확인한 후 실행해야 합니다.  

💻 **터미널 명령어:**  
```bash
python3.11 webscraping_movies.py
```

---

### **2️⃣ 예상 출력 결과**  
코드를 실행하면, 출력 결과는 다음과 같이 보일 것입니다:

<img src='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/images/output_screen.png' />

---

### **3️⃣ 생성된 파일 확인**  
코드 실행 후, **프로젝트 폴더**에 다음 두 파일이 생성됩니다:

1️⃣ **CSV 파일:**  
- `top_50_films.csv`  
- 이 파일은 상위 50개 영화 정보를 포함합니다.

2️⃣ **데이터베이스 파일:**  
- `Movies.db`  
- 데이터베이스 안에 **`Top_50`** 테이블이 생성되어, 영화 정보가 저장됩니다.

---

### **4️⃣ 작업 완료!** ✅  
코드를 성공적으로 실행하고, 파일이 생성되었는지 확인하세요.  

🎉 **축하합니다!** 모든 웹 스크래핑 작업을 성공적으로 완료했습니다.  
이제 **SQL 쿼리**를 사용하여 데이터베이스에서 정보를 검색할 수 있습니다! 🚀  

<div class='alert alert-block alert-warning'></div>

In [ ]:
import requests
import sqlite3
import pandas as pd
from bs4 import BeautifulSoup

url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'
db_name = 'Movies.db'
table_name = 'Top_50'
csv_path = 'top_50_films.csv'
df = pd.DataFrame(columns=["Average Rank","Film","Year"])
count = 0

html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')

tables = data.find_all('tbody')
rows = tables[0].find_all('tr')

for row in rows:
    if count < 50 :
        col = row.find_all('td')
        if len(col)!=0:
            data_dict = {"Average Rank": int(col[0].contents[0]),
                         "Film": str(col[1].contents[0]),
                         "Year": int(col[2].contents[0])}
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df,df1], ignore_index=True)
            count+=1
    else:
        break

print(df)

df.to_csv(csv_path)

conn = sqlite3.connect(db_name)
df.to_sql(table_name, conn, if_exists='replace', index=False)
conn.close()


---

## **실습 문제**  

아래 연습 문제를 통해 이번 실습에서 배운 내용을 복습해 보세요.  
해당 문제의 **정답은 제공되지 않으므로**, 필요할 경우 **토론 포럼**을 활용하여 도움을 받으세요.  

---

### **연습 문제 1: 코드 수정**  
✅ **수정 목표:**  
코드를 수정하여 **Film(영화 제목)**, **Year(제작 연도)**, 그리고 **Rotten Tomatoes' Top 100 헤더**를 추출하세요.  

💡 **힌트:**  
- HTML 구조에서 **새로운 헤더**를 확인하고 코드에서 데이터 딕셔너리에 추가하세요.  

---



In [22]:
import pandas as pd

url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'

tables = pd.read_html(url)

table_df = tables[1]

df = table_df.iloc[:,:4]
df.columns = ["Average Rank","Film","Year","Rotten Tomatoes' Top 100"]
df

,Average Rank,Film,Year,Rotten Tomatoes' Top 100
0,1,The Godfather,1972,17
1,2,Citizen Kane,1941,2
2,3,Casablanca,1942,8
3,4,"The Godfather, Part II",1974,99
4,5,Singin' in the Rain,1952,52
...,...,...,...,...
103,104,Titanic,1997,unranked
104,105,Toy Story,1995,unranked
105,106,Reservoir Dogs,1992,unranked
106,107,Paddington 2,2018,93


In [46]:
import requests
from bs4 import BeautifulSoup

url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'
df = pd.DataFrame(columns=["Average Rank","Film","Year","Rotten Tomatoes' Top 100"])
count = 0

html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')
tables = data.find_all('tbody')
rows = tables[0].find_all('tr')

for row in rows:
    if count < 50:
        
        col = row.find_all('td')
        if len(col) != 0:
            year = int(col[2].contents[0])  # 연도를 먼저 추출합니다.
            data_dict = {"Average Rank": int(col[0].contents[0]),
                         "Film": str(col[1].contents[0]),
                         "Year": year,
                         "Rotten Tomatoes' Top 100": col[3].contents[0],
                         }
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df, df1], ignore_index=True)
        count += 1
    else:
        break

In [48]:
df

,Average Rank,Film,Year,Rotten Tomatoes' Top 100
0,1,The Godfather,1972,17
1,2,Citizen Kane,1941,2
2,3,Casablanca,1942,8
3,4,"The Godfather, Part II",1974,99
4,5,Singin' in the Rain,1952,52
5,6,Psycho,1960,88
6,7,Rear Window,1954,47
7,8,Apocalypse Now,1979,unranked
8,9,2001: A Space Odyssey,1968,unranked
9,10,Seven Samurai,1954,49


### **연습 문제 2: 상위 25개 결과로 제한**  
✅ **수정 목표:**  
코드가 **상위 25개** 항목만 출력하도록 결과를 제한하세요.  

💡 **힌트:**  
- 루프 카운터 **`if count < 50`** 조건을 **`if count < 25`**로 변경하세요.  

---

In [49]:
import requests
from bs4 import BeautifulSoup

url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'
df = pd.DataFrame(columns=["Average Rank","Film","Year","Rotten Tomatoes' Top 100"])
count = 0

html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')
tables = data.find_all('tbody')
rows = tables[0].find_all('tr')

for row in rows:
    if count < 25:
        
        col = row.find_all('td')
        if len(col) != 0:
            year = int(col[2].contents[0])  # 연도를 먼저 추출합니다.
            data_dict = {"Average Rank": int(col[0].contents[0]),
                         "Film": str(col[1].contents[0]),
                         "Year": year,
                         "Rotten Tomatoes' Top 100": col[3].contents[0],
                         }
            df1 = pd.DataFrame(data_dict, index=[0])
            df = pd.concat([df, df1], ignore_index=True)
        count += 1
    else:
        break

In [50]:
df

,Average Rank,Film,Year,Rotten Tomatoes' Top 100
0,1,The Godfather,1972,17
1,2,Citizen Kane,1941,2
2,3,Casablanca,1942,8
3,4,"The Godfather, Part II",1974,99
4,5,Singin' in the Rain,1952,52
5,6,Psycho,1960,88
6,7,Rear Window,1954,47
7,8,Apocalypse Now,1979,unranked
8,9,2001: A Space Odyssey,1968,unranked
9,10,Seven Samurai,1954,49


### **연습 문제 3: 2000년대 개봉 영화만 출력**  
✅ **수정 목표:**  
출력 결과를 **2000년대(2000년~2009년)**에 개봉된 영화로 필터링하세요.  

💡 **힌트:**  
- 조건문을 추가하여 **`Year`** 컬럼의 값이 **2000 이상 2010 미만**인 경우만 출력하세요.

```python
if 2000 <= int(data_dict["Year"]) < 2010:
    print(data_dict)
```

---

In [51]:
import requests
from bs4 import BeautifulSoup

url = 'https://web.archive.org/web/20230902185655/https://en.everybodywiki.com/100_Most_Highly-Ranked_Films'
df = pd.DataFrame(columns=["Average Rank","Film","Year","Rotten Tomatoes' Top 100"])
count = 0

html_page = requests.get(url).text
data = BeautifulSoup(html_page, 'html.parser')
tables = data.find_all('tbody')
rows = tables[0].find_all('tr')

for row in rows:
    if count < 25:
        
        col = row.find_all('td')
        if len(col) != 0:
            year = int(col[2].contents[0])  # 연도를 먼저 추출합니다.
            if 2000 <= year < 2010:
                data_dict = {"Average Rank": int(col[0].contents[0]),
                             "Film": str(col[1].contents[0]),
                             "Year": year,
                             "Rotten Tomatoes' Top 100": col[3].contents[0],
                             }
                df1 = pd.DataFrame(data_dict, index=[0])
                df = pd.concat([df, df1], ignore_index=True)
        count += 1
    else:
        break

In [52]:
df

,Average Rank,Film,Year,Rotten Tomatoes' Top 100
0,19,Lord of the Rings: The Fellowship of the Ring,2001,unranked






### **코드 실행 후 결과 확인**  
모든 문제를 해결한 후, 코드를 실행하여 결과를 확인하세요:

```bash
python3.11 webscraping_movies.py
```

---

🎉 **연습 문제 완료 후, 자신감을 가져보세요!**  
웹 스크래핑과 데이터 추출 기술을 더 깊이 이해하게 될 것입니다. 🚀  
필요한 경우, 포럼에서 동료 학습자와 **아이디어를 공유**해 보세요!